# Figure 1

In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cmocean
import os
import cartopy.crs as ccrs
from matplotlib.patches import Rectangle
import matplotlib
import matplotlib.ticker as mticker
import geopandas as gpd
from shapely.geometry import box

data = xr.open_dataset('/mnt/d/Data/IBCSO_v2_bed_WGS84.nc').sel(lat=slice(-80, -55), lon=slice(0, 80))

shapefile_path = '/home/soumya/SOE2025/shapefile/antarctic_circumpolar_current_fronts.shp'
shape_data = gpd.read_file(shapefile_path)

# Clip the shapefile
bounding_box = box(20, -80, 90, -60)
clipped_shape = gpd.clip(shape_data, bounding_box)
sbf = clipped_shape[clipped_shape['DISPLAY'] == 'Southern boundary']

# Define the file path for Argo data
file_path = '/mnt/d/Data/Argo/Argo/PR_PF_7900334.csv'

# Extract the Argo float number from the filename
file_name = os.path.basename(file_path)
argo_float_number = file_name.split('_')[2].split('.')[0]

# Load the CSV file
df = pd.read_csv(file_path)

# Ensure the correct data types
df['DATE (YYYY-MM-DDTHH:MI:SSZ)'] = pd.to_datetime(df['DATE (YYYY-MM-DDTHH:MI:SSZ)'])
df['PRES (decibar)'] = pd.to_numeric(df['PRES (decibar)'], errors='coerce')
df['PSAL (psu)'] = pd.to_numeric(df['PSAL_ADJUSTED (psu)'], errors='coerce')
df['latitude'] = pd.to_numeric(df['LATITUDE (degree_north)'], errors='coerce')
df['longitude'] = pd.to_numeric(df['LONGITUDE (degree_east)'], errors='coerce')
df['POSITION_QC'] = pd.to_numeric(df['POSITION_QC'], errors='coerce')  # Ensure POSITION_QC is numeric

# Define the rectangle box coordinates
rect_min_lon, rect_max_lon = 20, 70
rect_min_lat, rect_max_lat = -70, -60

# Filter the Argo data based on the bounding box and date range
df = df[
        (df['longitude'] >= rect_min_lon) & 
        (df['longitude'] <= rect_max_lon) & 
        (df['latitude'] >= rect_min_lat) & 
        (df['latitude'] <= rect_max_lat)
    ]
df = df[(df['DATE (YYYY-MM-DDTHH:MI:SSZ)'] <= '2017-03-30')]

# Extract latitude, longitude, date, and QC flags of Argo profiles
argo_lats = df['latitude'].values
argo_lons = df['longitude'].values
argo_dates = df['DATE (YYYY-MM-DDTHH:MI:SSZ)']
qc_flags = df['POSITION_QC'].values

# Identify the first and last profiles
start_lat, start_lon, start_date = argo_lats[0], argo_lons[0], argo_dates.iloc[0]
end_lat, end_lon, end_date = argo_lats[-1], argo_lons[-1], argo_dates.iloc[-1]

# Define the specific date range for highlighting
start_date_highlight = '2016-08-08'
end_date_highlight = '2016-09-05'

# Filter Argo profiles within the highlight date range
highlight_profiles = df[(df['DATE (YYYY-MM-DDTHH:MI:SSZ)'] >= start_date_highlight) & 
                        (df['DATE (YYYY-MM-DDTHH:MI:SSZ)'] <= end_date_highlight)]

highlight_lats = highlight_profiles['latitude'].values
highlight_lons = highlight_profiles['longitude'].values

# Plotting Bathymetry with Argo Profiles
fig, ax = plt.subplots(1, 1, figsize=(20, 12), subplot_kw={'projection': ccrs.PlateCarree()})

# Custom colormap function
def custom_div_cmap(numcolors=11, name='custom_div_cmap',
                    mincol='blue', midcol='white', maxcol='red'):
    from matplotlib.colors import LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list(name=name, 
                                             colors =[mincol, midcol, maxcol],
                                             N=numcolors)
    return cmap

# Make negative contours solid
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

blevels = np.arange(-5000, 100, 100)
N = len(blevels) - 1
cmap2 = custom_div_cmap(N, mincol='DarkBlue', midcol='CornflowerBlue', maxcol='w')
cmap2.set_over('0.7')  # Light gray

# Plot filled contours of bathymetry
pc = ax.contourf(data['lon'], data['lat'], data['z'], levels=blevels, cmap=cmap2, extend='both')
cf = ax.contour(data['lon'], data['lat'], data['z'], levels=[-2000], colors=['magenta'], linewidths=2)


ds = xr.open_dataset('/mnt/d/Data/sea_ice_20022024_reprojected.nc').sel(x=slice(36, 60), y=slice(-63, -66))
sea_ice_concentration = ds['z']  # Adjust variable name as needed

# Calculate mean sea ice concentration for August 2016
sea_ice_seasonal = sea_ice_concentration.sel(time=slice('2016-08-11', '2016-08-12')).mean(dim='time')


# Plot contour lines for SIC (only 30% and optionally others)
contour = ax.contour(sea_ice_seasonal.x, sea_ice_seasonal.y, sea_ice_seasonal,
                     levels=[30], linewidths=2, colors='yellow')

sbf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=3, label='Southern Boundary Front', transform=ccrs.PlateCarree())
# Color mapping for POSITION_QC
color_map = {1: 'cyan', 8: 'red'}  # Green for QC=1, Purple for QC=8

# Plot Argo profiles with color based on POSITION_QC
for qc_value in df['POSITION_QC'].unique():
    subset = df[df['POSITION_QC'] == qc_value]
    ax.scatter(subset['longitude'], subset['latitude'], 
               color=color_map.get(qc_value, 'gray'),  # Default gray if unknown QC
               s=50, edgecolor='black', label=f'QC={qc_value}')

# Highlight specific profiles
ax.scatter(highlight_lons, highlight_lats, color='yellow', s=50, edgecolor='black', label='Highlighted Profiles')

# Add rectangle
rect = Rectangle((30, -67), 30, 6, linewidth=3, edgecolor='#17FF00', facecolor='none', transform=ccrs.PlateCarree())
ax.add_patch(rect)

# Annotate start and end points
ax.text(start_lon + 0.5, start_lat, f'Start ({start_date.strftime("%Y-%m-%d")})', fontsize=14, color='Red', weight='bold')
ax.text(end_lon - 7, end_lat, f'End ({end_date.strftime("%Y-%m-%d")})', fontsize=14, color='Red', weight='bold')

# Annotate geographical features
ax.text(53.5, -66.5, 'Cape \nAnn', fontsize=18, color='Black', weight='bold', ha='center', va='center')
ax.text(73, -69, 'Prydz \nBay', fontsize=18, color='Black', weight='bold', ha='center', va='center')

# Add coastlines and set extent
ax.coastlines(resolution='10m', linewidth=1)
ax.set_extent([25, 80, -70, -60], crs=ccrs.PlateCarree())
ax.set_aspect(3)

# Add gridlines and labels
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0)
gl.top_labels = False
gl.left_labels = True
gl.right_labels = False
gl.bottom_labels = True
gl.xlabel_style = {'size': 14}
gl.ylabel_style = {'size': 14}

# Add colorbar
cbar = plt.colorbar(pc, ax=ax, shrink=0.8, spacing='uniform')
cbar.ax.set_ylabel('Bathymetry (m)', fontsize=14)
cbar.ax.tick_params(labelsize=14)
plt.tight_layout()

plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/bathy.png', dpi=400)

# Figure 2 (a)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data
properties_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_30.csv')

# Filter data based on latitude and longitude ranges
properties_df = properties_df[(properties_df['lat'] >= -67) & (properties_df['lat'] <= -60) &
                              (properties_df['lon'] >= 30) & (properties_df['lon'] <= 60)]

# Convert 'Timestamp' column to datetime
properties_df['Timestamp'] = pd.to_datetime(properties_df['Timestamp'])

# Filter to include only June, July, August, and September
properties_df = properties_df[properties_df['Timestamp'].dt.month.isin([6, 7, 8, 9])]

# Group by month and calculate the sum and maximum of 'total_area'
properties_df['Month'] = properties_df['Timestamp'].dt.to_period('M')
monthly_area_stats = properties_df.groupby('Month')['total_area'].agg(['sum', 'max']).reset_index()

# Convert 'total_area' from square meters to square kilometers
monthly_area_stats['sum'] = monthly_area_stats['sum'] / (10**9)
monthly_area_stats['max'] = monthly_area_stats['max'] / (10**9)

# Sample data to show which years had wCSP or eCSP (customize this data as needed)
polynya_years = {
    'wCSP': ['2003-07', '2004-06', '2005-06', '2005-07', '2006-06', '2007-06', '2009-07', '2011-07', '2013-06',
             '2013-07', '2014-06', '2014-09', '2015-06', '2015-07', '2015-08', '2016-08', '2019-07', '2020-07',
             '2021-06', '2023-08', '2023-09'],
    'eCSP': ['2003-08', '2003-09', '2007-07', '2007-08', '2008-07', '2008-09', '2008-10', '2009-06', '2011-09',
             '2012-07', '2015-09', '2016-08', '2017-07', '2019-08', '2021-07', '2022-07', '2023-09']
}

# Add year column for easier tick labeling
monthly_area_stats['Year'] = monthly_area_stats['Month'].dt.year

# Plotting
fig, ax = plt.subplots(figsize=(20, 10))

# Bar width
bar_width = 0.8

# X locations for the groups
x = monthly_area_stats['Month'].astype(str)

# Plot bar for total area (sum)
ax.bar(x, monthly_area_stats['sum'], width=bar_width, color='skyblue', label='Total Area')

# Plot bar for maximum area
ax.bar(x, monthly_area_stats['max'], width=bar_width, color='salmon', label='Maximum Area')

# Add markers for polynya occurrences on top of the sum bars
for year in polynya_years['wCSP']:
    if year in x.values:
        ax.scatter(
            x[x == year],
            monthly_area_stats[monthly_area_stats['Month'].astype(str) == year]['sum'] - 40,
            color='green', s=100, marker='^', label='wCSP' if 'wCSP' not in ax.get_legend_handles_labels()[1] else ""
        )

for year in polynya_years['eCSP']:
    if year in x.values:
        ax.scatter(
            x[x == year],
            monthly_area_stats[monthly_area_stats['Month'].astype(str) == year]['sum'],
            color='brown', s=100, marker='D', label='eCSP' if 'eCSP' not in ax.get_legend_handles_labels()[1] else ""
        )

# Set labels and title
ax.set_ylabel('Polynya Area (x $10^{3}$ km²)', fontsize=16)
ax.tick_params(axis='x', labelsize=16, rotation=0)
ax.tick_params(axis='y', labelsize=16)

# Custom x-ticks: Use the first available month of each year
unique_years = monthly_area_stats['Year'].unique()
tick_positions = []
tick_labels = []

for year in unique_years:
    # Filter data for the specific year
    year_data = monthly_area_stats[monthly_area_stats['Year'] == year]
    
    # Get the first available month for the year
    first_month = year_data['Month'].min()  # Earliest month
    
    # Add tick position and label
    tick_positions.append(x.tolist().index(str(first_month)))
    tick_labels.append(year)

# Set the ticks and labels
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_labels, fontsize=16)


# Make plot frame bold by setting spines width
for spine in ax.spines.values():
    spine.set_linewidth(2.0)

# Adjust x-axis limits to remove extra space
ax.set_xlim(-0.5, len(x) - 0.5)

# Add legend
# ax.legend(fontsize=14)

plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/CSP_JJAS_yearly_ticks_total_and_max.png', bbox_inches='tight', dpi=300)
plt.show()


# Figure 2 (b)-(i)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
import matplotlib.ticker as mticker
from matplotlib.patches import Rectangle    

# Load the dataset
ds = xr.open_mfdataset('/mnt/d/Data/sea_ice_20022024.nc')

# Specify the exact days you want to plot
specific_days = ['2006-06-16', '2008-07-19', '2011-07-11', '2013-06-27', '2016-08-17', '2019-07-18', '2020-07-21', '2023-09-10']

# Extract data for the specified days
data = ds.sel(time=specific_days)
sea_ice_concentration = data['z']  # Adjust this based on your variable name

# Create the figure and 3x3 grid for the plot
fig, axes = plt.subplots(2, 4, figsize=(20, 10), subplot_kw={'projection': ccrs.SouthPolarStereo()}, constrained_layout=True)
axes = axes.flatten()

# Define the color levels for sea ice concentration
levels = np.arange(0, 101, 1)

for i, ax in enumerate(axes):
    if i < len(sea_ice_concentration.time):
        sea_ice_daily_avg = sea_ice_concentration.isel(time=i)
        contour = ax.contourf(sea_ice_daily_avg.x, sea_ice_daily_avg.y, sea_ice_daily_avg, levels=levels, cmap='coolwarm', extend='both')
        ax.coastlines(resolution='10m', linewidth=1)
        ax.add_feature(cfeature.LAND, facecolor='lightgray') 
        ax.set_extent([20, 80, -70, -60], crs=ccrs.PlateCarree())
        rect = Rectangle((30, -66), 30, 6, linewidth=3, edgecolor='#17FF00', facecolor='none', transform=ccrs.PlateCarree())
        ax.add_patch(rect)
        
        # Add longitude and latitude gridlines
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, linestyle='--')
        gl.xlocator = mticker.MultipleLocator(base=20)
        gl.ylocator = mticker.FixedLocator(range(-80, -59, 10))

        # Manually add latitude and longitude labels
        for lon in range(20, 81, 20):
            ax.text(lon, -65, f'{lon}°E', transform=ccrs.PlateCarree(), ha='center', va='top', fontsize=15)

        gl.top_labels = False
        gl.bottom_labels = False
        gl.left_labels = False
        gl.right_labels = False

        # Get the date for the current frame
        current_date = str(sea_ice_daily_avg.time.values)[:10]
        ax.text(0.62, 0.98, current_date, transform=ax.transAxes,
                fontsize=15, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
    else:
        ax.remove()  # Remove empty subplots

# plt.subplots_adjust(wspace=None, hspace=None)
# Adjust layout and add a colorbar
# cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
# cbar = fig.colorbar(contour, cax=cbar_ax, orientation='vertical')
# cbar.set_label('Sea Ice Concentration (%)')

# plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/evolution_diff_year.png', bbox_inches='tight' ,dpi=300)
plt.show()


# Figure S1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load polynya area data for different criteria
polynya_30_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_30.csv')
polynya_40_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_40.csv')
polynya_60_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_60.csv')

# Convert timestamp to datetime
polynya_30_df['Timestamp'] = pd.to_datetime(polynya_30_df['Timestamp'])
polynya_40_df['Timestamp'] = pd.to_datetime(polynya_40_df['Timestamp'])
polynya_60_df['Timestamp'] = pd.to_datetime(polynya_60_df['Timestamp'])

# Filter data for June to September for each year and region of interest
def filter_and_max_polynya_area(df):
    df = df[(df['Timestamp'].dt.month.isin([6, 7, 8, 9]))]
    df = df[(df['lat'] >= -67) & (df['lat'] <= -60) & (df['lon'] >= 30) & (df['lon'] <= 60)]
    df['Date'] = df['Timestamp'].dt.floor('D')
    daily_max = df.groupby('Date')['total_area'].sum().reset_index()
    daily_max['Year'] = daily_max['Date'].dt.year
    max_per_year = daily_max.loc[daily_max.groupby('Year')['total_area'].idxmax()]
    max_per_year['total_area'] = max_per_year['total_area'] / (10**9)  # Convert to square kilometers
    return max_per_year

# Get the maximum polynya area for each criterion
max_polynya_30 = filter_and_max_polynya_area(polynya_30_df)
max_polynya_40 = filter_and_max_polynya_area(polynya_40_df)
max_polynya_60 = filter_and_max_polynya_area(polynya_60_df)

# Merge data and sort by each year, so largest values are plotted first
merged_df = pd.concat([max_polynya_30.assign(Criterion='30'),
                       max_polynya_40.assign(Criterion='40'),
                       max_polynya_60.assign(Criterion='60')])

# Sort each year by descending polynya area for plotting order
merged_df = merged_df.sort_values(['Year', 'total_area'], ascending=[True, False])

# Plotting
fig, ax = plt.subplots(figsize=(20, 8))

# Set widths according to sorted values: largest gets widest, smallest gets narrowest
for year in merged_df['Year'].unique():
    year_data = merged_df[merged_df['Year'] == year]
    # Assign widths: 80 for largest, 65 for second, and 50 for smallest
    widths = [80, 65, 50]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Solid colors for each criterion
    
    for (index, row), width, color in zip(year_data.iterrows(), widths, colors):
        ax.bar(row['Date'], row['total_area'], color=color, width=width, label=f'Polynya Area ({row["Criterion"]}%)' if year == merged_df['Year'].min() else "")

# Format the x-axis for year and set limits from 2002 to 2023
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.set_xlim(pd.Timestamp('2002-01-01'), pd.Timestamp('2023-12-31'))
fig.autofmt_xdate(rotation=0)  # Rotate ticks 90 degrees for better readability

# Align ticks to the center of the markers
for label in ax.get_xticklabels():
    label.set_horizontalalignment('center')

# Add grid, labels, and title
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_xlabel('Year', fontsize=20)  # Increased font size for x-axis label
ax.set_ylabel('Polynya Area (x $10^{3}$ km²)', fontsize=20)  # Increased font size for y-axis label

# Enhance readability of ticks
ax.tick_params(axis='both', which='major', labelsize=16)

# Add legend with increased font size
# ax.legend(fontsize=18)

# Add title with larger font size
# plt.title('Comparison of Maximum Daily Polynya Area for different thresold (June–September)', fontsize=24)

plt.tight_layout()

# Save and display the plot
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/polynya_extent_comparison_solid_colors.png', dpi=400)
plt.show()


# Figure S2

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
import matplotlib.ticker as mticker
from matplotlib.patches import Rectangle

# Load the dataset for the specified time range
ds = xr.open_mfdataset('/mnt/d/Data/sea_ice_20022024.nc')

# Extract data for the specified time range
time_range = slice('2016-08-01', '2016-09-03')
data = ds.sel(time=time_range)
sea_ice_concentration = data['z']  # Adjust this based on your variable name

# Calculate the 3-day average
sea_ice_3day_avg = sea_ice_concentration.resample(time='3D').mean()

# Create the figure and axis for the plot
fig, axes = plt.subplots(4, 4, figsize=(20, 20), subplot_kw={'projection': ccrs.SouthPolarStereo()})
axes = axes.flatten()

# Define the color levels for sea ice concentration
levels = np.arange(0, 101, 1)

for i, ax in enumerate(axes):
    if i < len(sea_ice_3day_avg.time):
        sea_ice_daily_avg = sea_ice_3day_avg.isel(time=i)
        contour = ax.contourf(sea_ice_daily_avg.x, sea_ice_daily_avg.y, sea_ice_daily_avg, levels=levels, cmap='coolwarm', extend='both')
        ax.coastlines(resolution='10m', linewidth=1)
        ax.add_feature(cfeature.LAND, facecolor='lightgray')
        ax.set_extent([20, 80, -70, -60], crs=ccrs.PlateCarree())

        # Add longitude and latitude gridlines
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, linestyle='--')

        # Manually add latitude and longitude labels
        for lon in range(20, 81, 20):
            ax.text(lon, -65, f'{lon}°E', transform=ccrs.PlateCarree(), ha='center', va='top', fontsize=14)

        rect = Rectangle((30, -66), 30, 6, linewidth=3, edgecolor='#17FF00' , facecolor='none', transform=ccrs.PlateCarree())
        ax.add_patch(rect)
        # Add longitude and latitude gridlines
        gl.xlocator = mticker.MultipleLocator(base=20)
        gl.ylocator = mticker.FixedLocator(range(-80, -59, 10))

        gl.top_labels = False
        gl.bottom_labels = False
        gl.left_labels = False
        gl.right_labels = False

        # Add the date on the subplot
        current_date = str(sea_ice_daily_avg.time.values)[:10]
        ax.text(0.66, 0.98, current_date, transform=ax.transAxes,
                fontsize=14, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
    else:
        ax.remove()  # Remove empty subplots

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)

plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/evolution_3day_avg.png', dpi=400)
plt.show()
